# LNG Deal Economics

## A. LNG Deal Economics Calculation

### 1. Imports + Dataclasses

In [1]:
from dataclasses import dataclass, asdict
from typing import Literal, Dict, Any

In [2]:
@dataclass
class CargoParams:
    deal_type: Literal["FOB", "DES"]
    cargo_mmbtu: float
    sales_price_des: float
    purchase_price_fob: float
    boiloff_rate_voyage: float
    fuel_use_fraction_of_cargo: float
    freight_deduct_usd_per_mmbtu: float
    regas_fee_usd_per_mmbtu: float
    pipeline_tariff_usd_per_mmbtu: float
    hedge_price_usd_per_mmbtu: float
    hedge_volume_mmbtu: float


@dataclass
class ShippingParams:
    distance_nm: float
    speed_knots: float
    daily_charter_rate_usd: float
    boiloff_rate_sea_daily: float

### 2. Helper Functions

In [3]:
def calc_shipping_days(distance_nm: float, speed_knots: float) -> float:
    return distance_nm / (speed_knots * 24.0)


def calc_voyage_boiloff(cargo_mmbtu: float, shipping_days: float, daily_boiloff_rate: float) -> float:
    return cargo_mmbtu * daily_boiloff_rate * shipping_days


def calc_fuel_use(cargo_mmbtu: float, fuel_use_fraction: float) -> float:
    return cargo_mmbtu * fuel_use_fraction


def calc_freight_cost(daily_charter_rate_usd: float, shipping_days: float) -> float:
    return daily_charter_rate_usd * shipping_days


def calc_hedge_pnl(hedge_price: float, physical_price: float, hedge_volume: float) -> float:
    return (physical_price - hedge_price) * hedge_volume

### 3.  Core LNG P&L Function

In [4]:
def lng_cargo_economics(cargo: CargoParams, shipping: ShippingParams) -> Dict[str, Any]:

    shipping_days = calc_shipping_days(shipping.distance_nm, shipping.speed_knots)
    freight_cost_total = calc_freight_cost(shipping.daily_charter_rate_usd, shipping_days)

    voyage_boiloff_mmbtu = calc_voyage_boiloff(
        cargo.cargo_mmbtu, shipping_days, shipping.boiloff_rate_sea_daily
    )
    fuel_use_mmbtu = calc_fuel_use(cargo.cargo_mmbtu, cargo.fuel_use_fraction_of_cargo)

    total_losses_mmbtu = voyage_boiloff_mmbtu + fuel_use_mmbtu
    net_delivered_mmbtu = max(cargo.cargo_mmbtu - total_losses_mmbtu, 0)

    regas_cost_total = net_delivered_mmbtu * cargo.regas_fee_usd_per_mmbtu
    pipeline_cost_total = net_delivered_mmbtu * cargo.pipeline_tariff_usd_per_mmbtu

    if cargo.deal_type == "DES":
        fob_cost_total = cargo.cargo_mmbtu * cargo.purchase_price_fob
        des_revenue_total = net_delivered_mmbtu * cargo.sales_price_des

        gross_margin = des_revenue_total - fob_cost_total - freight_cost_total
        downstream_costs = regas_cost_total + pipeline_cost_total
        net_margin = gross_margin - downstream_costs

        physical_price_for_hedge = cargo.sales_price_des

    else:  # FOB
        fob_revenue_total = cargo.cargo_mmbtu * cargo.purchase_price_fob
        freight_deduct_total = cargo.cargo_mmbtu * cargo.freight_deduct_usd_per_mmbtu
        netback_total = fob_revenue_total - freight_deduct_total

        gross_margin = netback_total - freight_cost_total
        downstream_costs = regas_cost_total + pipeline_cost_total
        net_margin = gross_margin - downstream_costs

        physical_price_for_hedge = cargo.purchase_price_fob - cargo.freight_deduct_usd_per_mmbtu

    hedge_pnl = calc_hedge_pnl(
        hedge_price=cargo.hedge_price_usd_per_mmbtu,
        physical_price=physical_price_for_hedge,
        hedge_volume=cargo.hedge_volume_mmbtu
    )

    total_pnl = net_margin + hedge_pnl

    return {
        "shipping_days": round(shipping_days, 3),
        "voyage_boiloff_mmbtu": round(voyage_boiloff_mmbtu, 2),
        "fuel_use_mmbtu": round(fuel_use_mmbtu, 2),
        "total_losses_mmbtu": round(total_losses_mmbtu, 2),
        "net_delivered_mmbtu": round(net_delivered_mmbtu, 2),
        "freight_cost_total_usd": round(freight_cost_total, 2),
        "regas_cost_total_usd": round(regas_cost_total, 2),
        "pipeline_cost_total_usd": round(pipeline_cost_total, 2),
        "gross_margin_usd": round(gross_margin, 2),
        "downstream_costs_usd": round(downstream_costs, 2),
        "net_margin_usd": round(net_margin, 2),
        "hedge_pnl_usd": round(hedge_pnl, 2),
        "total_pnl_usd": round(total_pnl, 2),
    }

### 4. Example Scenario

In [5]:
cargo = CargoParams(
    deal_type="DES",
    cargo_mmbtu=3_000_000,
    sales_price_des=12,
    purchase_price_fob=9.5,
    boiloff_rate_voyage=0.001,
    fuel_use_fraction_of_cargo=0.02,
    freight_deduct_usd_per_mmbtu=0.20,
    regas_fee_usd_per_mmbtu=0.30,
    pipeline_tariff_usd_per_mmbtu=0.20,
    hedge_price_usd_per_mmbtu=11,
    hedge_volume_mmbtu=2_000_000
)

shipping = ShippingParams(
    distance_nm=9000,
    speed_knots=15,
    daily_charter_rate_usd=80000,
    boiloff_rate_sea_daily=0.001
)

result = lng_cargo_economics(cargo, shipping)
result

{'shipping_days': 25.0,
 'voyage_boiloff_mmbtu': 75000.0,
 'fuel_use_mmbtu': 60000.0,
 'total_losses_mmbtu': 135000.0,
 'net_delivered_mmbtu': 2865000.0,
 'freight_cost_total_usd': 2000000.0,
 'regas_cost_total_usd': 859500.0,
 'pipeline_cost_total_usd': 573000.0,
 'gross_margin_usd': 3880000.0,
 'downstream_costs_usd': 1432500.0,
 'net_margin_usd': 2447500.0,
 'hedge_pnl_usd': 2000000,
 'total_pnl_usd': 4447500.0}

In [6]:
result

{'shipping_days': 25.0,
 'voyage_boiloff_mmbtu': 75000.0,
 'fuel_use_mmbtu': 60000.0,
 'total_losses_mmbtu': 135000.0,
 'net_delivered_mmbtu': 2865000.0,
 'freight_cost_total_usd': 2000000.0,
 'regas_cost_total_usd': 859500.0,
 'pipeline_cost_total_usd': 573000.0,
 'gross_margin_usd': 3880000.0,
 'downstream_costs_usd': 1432500.0,
 'net_margin_usd': 2447500.0,
 'hedge_pnl_usd': 2000000,
 'total_pnl_usd': 4447500.0}

# B. Sensitivity Tables

## 1. DES Price vs Total P&L

In [7]:
import numpy as np
import pandas as pd

des_prices = np.linspace(8, 20, 25)  # range of DES prices
pnl_list = []

for p in des_prices:
    cargo.sales_price_des = p
    result = lng_cargo_economics(cargo, shipping)
    pnl_list.append(result["total_pnl_usd"])

df_des_pnl = pd.DataFrame({
    "DES Price (USD/MMBtu)": des_prices,
    "Total P&L (USD)": pnl_list
})

df_des_pnl

,DES Price (USD/MMBtu),Total P&L (USD)
0,8.0,-15012500.0
1,8.5,-12580000.0
2,9.0,-10147500.0
3,9.5,-7715000.0
4,10.0,-5282500.0
5,10.5,-2850000.0
6,11.0,-417500.0
7,11.5,2015000.0
8,12.0,4447500.0
9,12.5,6880000.0


## 2. Freight vs Netback

In [8]:
freight_values = np.linspace(0.1, 2.0, 25)
netback_list = []

for f in freight_values:
    cargo.freight_deduct_usd_per_mmbtu = f
    result = lng_cargo_economics(cargo, shipping)
    netback_list.append(result["net_margin_usd"])

df_freight_netback = pd.DataFrame({
    "Freight Deduct (USD/MMBtu)": freight_values,
    "Netback (USD)": netback_list
})

df_freight_netback

,Freight Deduct (USD/MMBtu),Netback (USD)
0,0.100000,25367500.0
1,0.179167,25367500.0
2,0.258333,25367500.0
3,0.337500,25367500.0
4,0.416667,25367500.0
5,0.495833,25367500.0
6,0.575000,25367500.0
7,0.654167,25367500.0
8,0.733333,25367500.0
9,0.812500,25367500.0


## 3. Boil‑off vs Delivered Volume

In [9]:
boiloff_values = np.linspace(0.0005, 0.005, 25)
delivered_list = []

for b in boiloff_values:
    shipping.boiloff_rate_sea_daily = b
    result = lng_cargo_economics(cargo, shipping)
    delivered_list.append(result["net_delivered_mmbtu"])

df_boiloff_delivery = pd.DataFrame({
    "Daily Boiloff Rate": boiloff_values,
    "Delivered Volume (MMBtu)": delivered_list
})

df_boiloff_delivery

,Daily Boiloff Rate,Delivered Volume (MMBtu)
0,0.000500,2902500.0
1,0.000688,2888437.5
2,0.000875,2874375.0
3,0.001063,2860312.5
4,0.001250,2846250.0
5,0.001438,2832187.5
6,0.001625,2818125.0
7,0.001813,2804062.5
8,0.002000,2790000.0
9,0.002188,2775937.5


# C. Plotly Interactive Charts

## 1. DES Price vs Total P&L Curve

In [10]:
!pip install plotly

In [11]:
import plotly.express as px

fig = px.line(
    df_des_pnl,
    x="DES Price (USD/MMBtu)",
    y="Total P&L (USD)",
    title="DES Price vs Total P&L"
)
fig.show()

## 2. Freight vs Netback Curve

In [12]:
fig = px.line(
    df_freight_netback,
    x="Freight Deduct (USD/MMBtu)",
    y="Netback (USD)",
    title="Freight Deduct vs Netback"
)
fig.show()

## 3. Boil‑off vs Delivered Volume Curve

In [13]:
fig = px.line(
    df_boiloff_delivery,
    x="Daily Boiloff Rate",
    y="Delivered Volume (MMBtu)",
    title="Boiloff Rate vs Delivered Volume"
)
fig.show()

## D. Interactive Plotly Sliders (ipywidgets)

## 1. Interactive DES Price Slider

In [14]:
import ipywidgets as widgets
from ipywidgets import interact

@interact(des_price=widgets.FloatSlider(min=8, max=20, step=0.1, value=12))
def interactive_des(des_price):
    cargo.sales_price_des = des_price
    result = lng_cargo_economics(cargo, shipping)
    print(f"DES Price: {des_price}")
    print(f"Total P&L: {result['total_pnl_usd']:,}")

interactive(children=(FloatSlider(value=12.0, description='des_price', max=20.0, min=8.0), Output()), _dom_cla…

## 2. Interactive Hedge P&L Visualisation

In [15]:
@interact(
    hedge_price=widgets.FloatSlider(min=5, max=20, step=0.1, value=11),
    hedge_volume=widgets.FloatSlider(min=0, max=3_000_000, step=50_000, value=2_000_000)
)
def hedge_visual(hedge_price, hedge_volume):
    cargo.hedge_price_usd_per_mmbtu = hedge_price
    cargo.hedge_volume_mmbtu = hedge_volume
    result = lng_cargo_economics(cargo, shipping)
    print(f"Hedge Price: {hedge_price}")
    print(f"Hedge Volume: {hedge_volume:,}")
    print(f"Hedge P&L: {result['hedge_pnl_usd']:,}")
    print(f"Total P&L: {result['total_pnl_usd']:,}")

interactive(children=(FloatSlider(value=11.0, description='hedge_price', max=20.0, min=5.0), FloatSlider(value…

# D. Margin Curve with Interactive DES Price Slider

In [16]:
@interact(des_price=widgets.FloatSlider(min=8, max=20, step=0.1, value=12))
def margin_curve(des_price):
    cargo.sales_price_des = des_price
    result = lng_cargo_economics(cargo, shipping)

    fig = px.bar(
        x=["Gross Margin", "Net Margin", "Total P&L"],
        y=[result["gross_margin_usd"], result["net_margin_usd"], result["total_pnl_usd"]],
        title=f"Margin Breakdown at DES Price = {des_price}"
    )
    fig.show()

interactive(children=(FloatSlider(value=12.0, description='des_price', max=20.0, min=8.0), Output()), _dom_cla…